### 3. TensorFlow 기본

#### 3.1 텐서와 그래프 실행

In [1]:
import tensorflow as tf

# tf.constant : 텐서플로 상수.
hello = tf.constant('Hello, TensorFlow!')
print(hello)

a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a, b)  # a + b로도 쓸 수 있다.
print(c)

Tensor("Const:0", shape=(), dtype=string)
Tensor("Add:0", shape=(), dtype=int32)



hello가 텐서플로의 **텐서**라는 자료형이고, 상수를 담는다.

텐서는 텐서플로에서 다양한 수학식을 계산하기 위한 가장 기본적이고 중요한 자료형이며 **랭크**와 **셰이프**라는 개념을 가지고있다.

* 3 : 랭크가 0인 텐서; 셰이프는 []
* [1. ,2. ,3.] : 랭크가 1인 텐서; 셰이프는 [3]
* [[1., 2., 3.], [4., 5., 6.]] : 랭크가 2인 텐서; 셰이프는 [2, 3]
* [[[1., 2., 3.], [4., 5., 6.], [7., 8., 9.]]] : 랭크가 3인 텐서; 셰이프는 [2, 1, 3]

텐서 자료형의 형태는 배열과 비슷하다. **랭크**는 차원의 수이다. 
* 랭크 0 : 스칼라
* 랭크 1 : 벡터
* 랭크 2 : 행렬
* 랭크3 : n-TensorFlow, n차원 텐서
**셰이프**는 각 차원의 요소 개수

텐서를 출력할 때 나오는 **dtype은 해당 텐서에 담긴 요소들의 자료형**이다. string, float, int등이 올 수 있다.


위에서 변수와 수식들을 정의해씨만, 실행이 정의한 시점에서 실행되는 것은 아니다.

그 이유는 텐서플로 프로그램 구조가 
1. 그래프 생성
2. 그래프 실행

두 가지로 분리되어 있기 때문이다.

![텐서플로 구조](./img/img1.JPG)

**그래프**는 텐서들의 연산 모음이다. 

텐서플로는 (1)텐서와 텐서의 연산들을 먼저 정의하여 그래프를 만들고
(2) 이후 필요할 때 연산을 실행하는 코드를 넣어 *'원하는 시점'*에 실제 연산을 수행하도록 한다. 
이러한 방식은 **지연 실행**이라하며 함수형 프로그래밍에서 많이 사용한다.

다음처럼 Session 객체와 run 메소드를 사용할 때 계산이 된다.

따라서 모델을 구성하는 것과, 실행하는 것을 분리하여 프로그램을 깔끔하게 작성할 수 있다.

In [2]:
# 그래프를 실행할 세션을 구성한다.
sess = tf.Session()

# sess.run : 설정한 텐서 그래프(변수나 수식)을 실행한다.
print(sess.run(hello))
print(sess.run([a, b, c]))

# 세션을 닫는다
sess.close()

b'Hello, TensorFlow!'
[10, 32, 42]


그래프의 실행은 Session안에서 이루어져야 하며, session객체와 run 메소드를 사용한다.


#### 3.2 플레이스홀더와 변수
* 플레이스폴더

   그래프에 사용할 입력값을 나중에 받기 위해 사용하는 매개변수
   
   !. 변수는 그래프를 최적화하는 용도로 텐서플로가(학습함수들이) 학습한 결과를 갱신하기 위해 사용하는 변수이다.
   

In [3]:
# None은 크기가 정해지지 않았음을 의미
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder:0", shape=(?, 3), dtype=float32)


float32 자료형을 가진 텐서가 생성됐다.

나중에 플레이스홀더 X에 넣을 자료를 다음과 같이 정의해 볼 수 있다.

앞서 텐서 모양을 (?, 3)으로 정의했으므로, 두 번째 차원은 요소를 3개씩 가지고 있어야 한다.

In [4]:
x_data = [[1,2,3], [4,5,6]]

#변수 정의
W = tf.Variable(tf.random_normal([3, 2]))  # random_normal : 정규분포의 무작위 값
b = tf.Variable(tf.random_normal([2, 1]))


각각 W와 b에 텐서플로 변수를 생성하여 할당한다.

W는 [3, 2] 행렬형태의 텐서, b는 [2, 1] 행렬 형대의 텐서로 정규분포의 무작위 값으로 초기화했다.

물론 다른 함수를 쓰거나 특정 형태의 데이터를 만들어서 넣어줄 수도 있다.


다음으로 입력값과 변수들을 계산할 수식을 작성한다.

X와 W가 행렬이기 때문에 marmul 함수를 사용해햐 한다.

*행렬이 아닌 경우에는 단순히 곱셈 연산자나 mul함수를 사용*


In [5]:
expr = tf.matmul(X, W) + b


**행렬곱 매우 중요**

![행렬곱](./img/img2.JPG)

위 정의에 따라 앞서 X에 넣을 데이터를 [2,3] 형태로 정의하였으므로 행렬곱을 위해 W의 형태를 [3,2]로 정의한 것.

이제 연산을 실행하고 결과를 출력하여, 설정한 텐서들과 계산됭 그래프의 결과를 확인한다.

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())   # 앞서 정의한 변수들을 초기화하는 함수.

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
print(sess.run(expr, feed_dict={X: x_data}))    # feed_dict : 그래프를 실행할 때 사용할 입력값을 지정한다.

sess.close()

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[-0.88866204 -0.09361325]
 [ 0.35012633  0.88960224]
 [ 0.550321   -0.6291377 ]]
=== b ===
[[ 2.053232]
 [-1.285216]]
=== expr ===
[[ 3.5157855   1.8514102 ]
 [ 0.21269333 -0.9864837 ]]



global_variables_initializer : 기존에 학습한 값을 가져와서 사용하는 것이 아닌 처음 실행하는 것이라면 연산을 실행하기 전에
    반드시 이 함수를 이용해 변수들을 초기화해야 한다.
    
feed_dict : expr 수식에는 X, W, b를 사용했는데, 이 중 X가 플레이스홀더라 X에 값을 넣어주지 않으면 계산에 사용할 수 없기 때문에
    에러가 난다. 따라서 미리 정의해둔 x_data를 X의 값으로 넣어준다. ***딕셔너리여도 되나??***
    

#### 3.3 선형 회귀 모델 구현
**선형 회귀**란 주어진 x와 y값을 가지고 서로 간의 관계를 파악하는 것

이 관계를 알고나면 새로운 x가 주어졌을 때 y의 값을 쉽게 알 수 있다. 

![선형 회귀 그래프](./img/img3.JPG)


이제 텐서플로의 최적화 함수를 이용해 X와 Y의 상관관계를 분석하는 기초적인 선형 회귀 모델을 만들고 실행해본다.

다음과 같이 주어진 x_data와 y_data의 상관관계를 파악해보자.


In [7]:

x_data = [1, 2, 3]
y_data = [1, 2, 3]


# x와 y의 상관관계를 설명하기 위한 변수들인 W, b를 
# 각각 -0.1 ~ 1.0 사이의 균등분포를 가진 무작위값으로 초기화한다.
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

# 자료를 입력받을 플레이스홀더 설정
# name: 나중에 텐서보드등으로 값의 변화를 추적하거나 살펴보기 쉽게 하기 위해 이름을 붙여줍니다.
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

# X와 Y의 상관관계(여기서는 선형관계)를 분석하기 위한 수실을 작성한다.
hypothesis = W * X + b    

위 수식은 W와의 곱과 b와의 합을 통해 X와 Y의 관계를 설명하겠다는 뜻이다.

**다시말해서 X가 주어졌을 때 Y를 만들어 낼 수 있는 W와 b를 찾아내겠다는 의미.**

W는 **가중치**, b는 **편향**이라고 하고, 위 수식은 신경망 학습에서 가장 기본이 되는 수식이다.
여기서는 w와 X가 행렬이 아니므로 tf.matmul 함수가 아니라 곱셈 연산자(*)를 사용했다.


In [8]:
# 손실 함수를 작성한다.
# mean(h - Y)^2 : 예측값과 실제값의 거리를 비용(손실) 함수로 정합니다.
cost = tf.reduce_mean(tf.square(hypothesis - Y))


**손실 함수(loss function)**는 한 쌍(x,y)의 데이터에 대한 손실값을 계산하는 함수이다.

손실값이란 실제값과 모델로 예측한 가설의 값이 얼마나 차이가 나는가를 나타내는 값이다.

즉, 손실값이 작을수록 그 모델이 X와 Y의 관계를 잘 설명하고 있는다는 뜻으로 *X값에 대한 Y값을 정확하게 예측할 수 있다*는 뜻이 된다.

이 손실을 전체 데이터에 대해 구한 경우 이를 **비용**이라고 한다.

즉, **학습**이란 변수들의 값을 다양하게 넣어 계산해보면서 이 **손실값을 최소화하는 W와 b의 값을 구하는 것**이다.


손실값으로는 '예측값과 실제값의 거리'를 가장 많이 사용한다. 

손실값은 예측값에서 실제값을 뺀 뒤 제곱하는데, 비용은 모든 데이터에 대한 손실값의 평균을 내어 구한다.

  
마지막으로 경사하강법 최적화 함수를 이용해 손실값을 최소화하는 연산 그래프를 생성한다.

In [9]:

# 경사하강법 최적화 함수를 이용해 손실값을 최소화하는 연산 그래프 생성
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_op = optimizer.minimize(cost)

**최적화함수**란 가중치와 편향의 값을 변경해가면서 손실값을 최소화하는 *가장 최적화된 가중치와 편향 값을 찾아주는 함수*이다.

이때 값들을 무작위로 변경하면 시간이 오래걸리고 예측도 쉽지 않다. 따라서 빠르게 최적화하기 위한 다양한 방법을 사용한다.

경사하강법은 최적화 방법 중 가장 기본적인 알고리즘으로,

![경사하강법](./img/img4.JPG)  
함수의 기울기를 구하고 기울기가 낮은 족으로 계속 이동시키면서 최적의 값을 찾아가는 방법이다.

매개변수인 learning_rate는 **학습률**로 학습을 얼마나 "급하게"할 것인가를 설정하는 값이다. 값이 너무 크거나 작으면 좋지 않다.
(크면 최적의 값을 찾지 못하고, 작으면 속도가 느리다)

이렇게 학습 진행 과정에 영향을 주는 변수를 **하이퍼파라미터**라고 하며, 이 값에 따라 속도나 성능이 크게 달라질 수 있다.

이제 모델은 다 만들었고, 그래프를 실행해 학습시키고 결과를 확인한다.


In [17]:
# 세션은 생성하고 변수들을 초기화 한다. 
# 파이썬 with 기능을 이용해 세션 블록을 만들고 종료를 자동으로 처리하도록 했다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # 최적화를 수행하는 그래프인 train_op를 실행하고
    # 실행 시마다 변화하는 손실값을 출력하는 코드
    # 학습은 100번 수행하며 feed_dict 매개변수로 상관관계를 알아내고자 하는 x_data, y_data를 입력해준다.
    for step in range(100):
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})

        print(step, cost_val, sess.run(W), sess.run(b))
        
    # 최적화가 완료된 모델에 테스트 값을 넣고 결과가 나오는지 확인
    print("\n=====Test=========")
    print("X: 5, Y:", sess.run(hypothesis, feed_dict={X: 5}))
    print("X: 2.5, Y:", sess.run(hypothesis, feed_dict={X: 2.5}))


0 0.27773085 [0.5816037] [0.9928714]
1 0.14106424 [0.5749584] [0.9616557]
2 0.13288867 [0.5870016] [0.9393412]
3 0.12655877 [0.5967303] [0.9166723]
4 0.1205469 [0.60644644] [0.8946457]
5 0.11482086 [0.6159048] [0.87313795]
6 0.10936674 [0.62513846] [0.8521484]
7 0.1041717 [0.63414985] [0.83166337]
8 0.09922349 [0.64294463] [0.8116707]
9 0.09451029 [0.651528] [0.7921587]
10 0.090021014 [0.659905] [0.77311575]
11 0.08574492 [0.6680807] [0.7545306]
12 0.08167198 [0.6760598] [0.7363922]
13 0.07779249 [0.68384707] [0.71868986]
14 0.07409727 [0.69144714] [0.70141304]
15 0.07057763 [0.6988646] [0.6845516]
16 0.067225106 [0.7061036] [0.6680954]
17 0.06403191 [0.71316874] [0.6520349]
18 0.060990345 [0.7200639] [0.6363604]
19 0.058093254 [0.7267934] [0.62106276]
20 0.055333775 [0.7333611] [0.6061328]
21 0.05270542 [0.739771] [0.5915618]
22 0.050201822 [0.74602664] [0.577341]
23 0.047817186 [0.752132] [0.56346214]
24 0.04554588 [0.7580906] [0.5499169]
25 0.043382373 [0.7639059] [0.53669727]
26 0.

손실값과 변수들의 변화를 확인 할 수 있다.

손실값이 점점 줄어든다면 학습이 정상적으로 이뤄지고있는 것이다.

*** _ 는 뭐지